# Метод аддитивной оптимизации

In [2]:
import pandas as pd
import sys
if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO
import numpy as np

## Заведение исходных данных

In [16]:
weights = pd.read_csv('Искомый коэффициент значимости.csv',sep=';').set_index('Unnamed: 0').T
weights

Unnamed: 0,стоимость (тыс. руб.),материал стен,Район,площадь (м2),этаж,количество комнат,Сумма
Искомый коэффициент значимости,0.247619,0.195238,0.22381,0.185714,0.080952,0.066667,1.0


In [31]:
weightsArray = [weights['Район'], weights['количество комнат'], weights['площадь (м2)'],
 weights['этаж'], weights['материал стен'], weights['стоимость (тыс. руб.)']]
pd.DataFrame(weightsArray)[]

,Искомый коэффициент значимости
Район,0.223810
количество комнат,0.066667
площадь (м2),0.185714
этаж,0.080952
материал стен,0.195238
стоимость (тыс. руб.),0.247619


In [33]:
data = StringIO('''Мотоцикл;Цена, руб;Год выпуска;Пробег, км;Объем двигателя, см3;Мощность двигателя, л.с. 
BMW R Nine T;1099000.00;2017;1070;1200;125 
Harley-Davidson V-Rod;5300000.00;2015;1;1250;209 
Ducati 1299 Panigale;1399000.00;2016;1600;1299;205 
BMW R 1200 GS;1305000.00;2017;4583;1170;125 
Triumph Tiger Explorer;1270000.00;2018;2900;1200;141
Веса критериев;0.33;0.07;0.25;0.13;0.22''')
data = StringIO('''Адрес;Район;Количество комнат;Площадь, м2;Этаж;Материал;Стоимость, тыс.руб.
Каслинская ул, д. 99в;Калининский;3;100;3/11;Кирпич;8600
Шагольская ул, д. 34;Курчатовский;4;89;4/5;Кирпич;2499
Шагольская ул, д. 41;Курчатовский;3;65;2/5;Панельный;2150
Прокатная ул, д. 26;Металлургический;3;91;1/16;Панельный;2250
Свободы ул, д. 88б;Советский;3;50;6/6;Кирпич;4550
Лобинский 10-й пер, д. 3;Тракторозаводский;3;63;4/5;Панельный;2180
МОПРа площадь ул, д. 9;Центральный;4;198.1;9/16;Кирпич;8000''')
df = pd.read_csv(data, sep=";").set_index('Адрес')
originalDataLen = len(df)
df.loc['Веса критериев'] = pd.DataFrame(weightsArray)
df.head(originalDataLen+10)

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

## Определим максимум и минимум каждого локального критерия

In [3]:
df.loc['max'] = (df.head(5).max(axis=0))
df.loc['min'] = (df.head(5).min(axis=0))

In [7]:
df.head(3+originalDataLen)

,Район,Количество комнат,"Площадь, м2",Этаж,Материал,"Стоимость, тыс.руб."
Адрес,,,,,,
"Каслинская ул, д. 99в",Калининский,3,100.0,3/11,Кирпич,8600
"Шагольская ул, д. 34",Курчатовский,4,89.0,4/5,Кирпич,2499
"Шагольская ул, д. 41",Курчатовский,3,65.0,2/5,Панельный,2150
"Прокатная ул, д. 26",Металлургический,3,91.0,1/16,Панельный,2250
"Свободы ул, д. 88б",Советский,3,50.0,6/6,Кирпич,4550
"Лобинский 10-й пер, д. 3",Тракторозаводский,3,63.0,4/5,Панельный,2180
"МОПРа площадь ул, д. 9",Центральный,4,198.1,9/16,Кирпич,8000


### Выбор критериев которые максимизируем и минимизируем  
  
Правило выбора таково:  
     1. если критерий имеет прямую зависимость с качеством товара <b>(критерий растет - качество растет), то МАКСИМИЗИРУЕМ</b>  
     2. если критерий имеет обратную зависимость с качеством товара <b>(критерий падает - качество растёт), то МИНИМИЗИРУЕМ</b>  

In [5]:
maximizationMinimizationParametrDict = {}
print('"+" - максимизируем; "-" - минимизируем.')
for columnName in df.columns:
    maximizationMinimizationParametrDict[columnName] = \
        input(f'Критерий: "{columnName}", введите +/-: ')

"+" - максимизируем; "-" - минимизируем.
Критерий: "Цена, руб", введите +/-: -
Критерий: "Год выпуска", введите +/-: -
Критерий: "Пробег, км", введите +/-: -
Критерий: "Объем двигателя, см3", введите +/-: +
Критерий: "Мощность двигателя, л.с. ", введите +/-: +


In [6]:
maximizationMinimizationParametrDict

{'Цена, руб': '-',
 'Год выпуска': '-',
 'Пробег, км': '-',
 'Объем двигателя, см3': '+',
 'Мощность двигателя, л.с. ': '+'}

## Максимизируем/минимизируем столбцы двумя типами

In [7]:
normalizedType1DF = df.copy()
normalizedType2DF = df.copy()
for columnName in df.columns:
    
    denominator = df[columnName]['max'] - df[columnName]['min']
    
    if maximizationMinimizationParametrDict[columnName] == '+':
        normalizedType1DF[columnName].iloc[0:originalDataLen] = \
            df[columnName].iloc[0:originalDataLen] / df[columnName]['max']
        
        normalizedType2DF[columnName].iloc[0:originalDataLen] = \
            (df[columnName].iloc[0:originalDataLen] - df[columnName]['min']) / denominator
    else:
        normalizedType1DF[columnName].iloc[0:originalDataLen] = \
            1 - df[columnName].iloc[0:originalDataLen] / df[columnName]['max']
        
        normalizedType2DF[columnName].iloc[0:originalDataLen] = \
            (df[columnName]['max'] - df[columnName].iloc[0:originalDataLen]) / denominator


### Типом 1: текущее / максимальное; 1 - текущее / максимальное

In [8]:
normalizedType1DF.head(originalDataLen+1)

,"Цена, руб",Год выпуска,"Пробег, км","Объем двигателя, см3","Мощность двигателя, л.с."
Мотоцикл,,,,,
BMW R Nine T,0.792642,0.000496,0.766528,0.923788,0.598086
Harley-Davidson V-Rod,0.000000,0.001487,0.999782,0.962279,1.000000
Ducati 1299 Panigale,0.736038,0.000991,0.650884,1.000000,0.980861
BMW R 1200 GS,0.753774,0.000496,0.000000,0.900693,0.598086
Triumph Tiger Explorer,0.760377,0.000000,0.367227,0.923788,0.674641
Веса критериев,0.330000,0.070000,0.250000,0.130000,0.220000


### Типом 2: ( текущее - мин ) / ( макс - мин ); ( макс - текущее  ) / ( макс - мин ) 

In [9]:
normalizedType2DF.head(6)

,"Цена, руб",Год выпуска,"Пробег, км","Объем двигателя, см3","Мощность двигателя, л.с."
Мотоцикл,,,,,
BMW R Nine T,1.000000,0.333333,0.766696,0.232558,0.000000
Harley-Davidson V-Rod,0.000000,1.000000,1.000000,0.620155,1.000000
Ducati 1299 Panigale,0.928588,0.666667,0.651026,1.000000,0.952381
BMW R 1200 GS,0.950964,0.333333,0.000000,0.000000,0.000000
Triumph Tiger Explorer,0.959295,0.000000,0.367307,0.232558,0.190476
Веса критериев,0.330000,0.070000,0.250000,0.130000,0.220000


In [10]:
normalizedType1DF['Обобщенная функ. цели Fi'] = \
    (normalizedType1DF[:][0:originalDataLen]*normalizedType1DF[:].loc['Веса критериев']).sum(axis=1)
print('Fi max =',normalizedType1DF['Обобщенная функ. цели Fi'].max())

Fi max = 0.7514722272920178


In [11]:
normalizedType1DF.head(originalDataLen+1)

,"Цена, руб",Год выпуска,"Пробег, км","Объем двигателя, см3","Мощность двигателя, л.с.",Обобщенная функ. цели Fi
Мотоцикл,,,,,,
BMW R Nine T,0.792642,0.000496,0.766528,0.923788,0.598086,0.704910
Harley-Davidson V-Rod,0.000000,0.001487,0.999782,0.962279,1.000000,0.595146
Ducati 1299 Panigale,0.736038,0.000991,0.650884,1.000000,0.980861,0.751472
BMW R 1200 GS,0.753774,0.000496,0.000000,0.900693,0.598086,0.497449
Triumph Tiger Explorer,0.760377,0.000000,0.367227,0.923788,0.674641,0.611245
Веса критериев,0.330000,0.070000,0.250000,0.130000,0.220000,NaN


In [12]:
normalizedType2DF['Обобщенная функ. цели Fi'] = \
    (normalizedType2DF[:][0:originalDataLen]*normalizedType2DF[:].loc['Веса критериев']).sum(axis=1)
print('Fi max =',normalizedType2DF['Обобщенная функ. цели Fi'].max())

Fi max = 0.8553810967645927


In [13]:
normalizedType2DF.head(originalDataLen+1)

,"Цена, руб",Год выпуска,"Пробег, км","Объем двигателя, см3","Мощность двигателя, л.с.",Обобщенная функ. цели Fi
Мотоцикл,,,,,,
BMW R Nine T,1.000000,0.333333,0.766696,0.232558,0.000000,0.575240
Harley-Davidson V-Rod,0.000000,1.000000,1.000000,0.620155,1.000000,0.620620
Ducati 1299 Panigale,0.928588,0.666667,0.651026,1.000000,0.952381,0.855381
BMW R 1200 GS,0.950964,0.333333,0.000000,0.000000,0.000000,0.337151
Triumph Tiger Explorer,0.959295,0.000000,0.367307,0.232558,0.190476,0.480532
Веса критериев,0.330000,0.070000,0.250000,0.130000,0.220000,NaN
